In [1]:
from sentence_transformers import SentenceTransformer, util
import torch
import numpy as np
import pandas as pd
import os
import pickle
import time
import re

/home/omar/Desktop/Logatta/SearchEngine/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = SentenceTransformer('all-MiniLM-L6-v2')


In [3]:
# import data
df = pd.read_csv('myDatabase.facebook_posts.csv')

In [4]:
df.isnull().sum()


_id              0
Post_Content    22
dtype: int64

In [5]:
# drop rows with null values
df = df.dropna()

In [8]:
df.head()

,_id,Post_Content
0,{'$oid': '645b5de694a0b0947d4ead81'},وسادة الحمل والرضاعة الحجم الكبير 180*80 ب سع...
1,{'$oid': '645b5de694a0b0947d4ead82'},البلف المضيء ل الاربع كوشوكات ب سعر 4.99 دناني...
2,{'$oid': '645b5de694a0b0947d4ead83'},الة تسليك المجاري وعلاج انسداد مواسير الصرف ال...
3,{'$oid': '645b5de694a0b0947d4ead84'},مرآة تجميل X10 مع لمبة ليد مريحة للاستخدام-مرن...
4,{'$oid': '645b5de694a0b0947d4ead85'},تعبئة فراغات الشعر متوفر لون اسود ولون بنيالعب...


In [9]:
ids = df['_id'].tolist()
names = df['Post_Content'].tolist()

In [10]:
# for name in names:
#     name = re.sub(r'[^\w\s]', '', name)
#     name = name.lower()

In [10]:
# Embed the dataset 
name_embeddings = model.encode(names, convert_to_tensor=True)

In [11]:
name_embeddings.shape

torch.Size([21134, 384])

In [12]:
def semantic_search(query, embeddings, texts, ids, top_k=100):
    # Encode the query
    query_embedding = model.encode(query, convert_to_tensor=True)
  
    # Compute cosine similarity between the query and texts
    cos_similarities = util.cos_sim(query_embedding, embeddings)
  
    # Sort the results by cosine similarity in descending order
    sorted_indices = cos_similarities.argsort(descending=True)
  
    # Return the top-k most similar texts and their corresponding indices
    top_k_indices = sorted_indices[0][:top_k]
    return [(texts[i], ids[i]) for i in top_k_indices]


In [16]:
query = input('Enter your query: ')
results = semantic_search(query, name_embeddings, names, ids, top_k=100)
for name, idx in results:
    print(f'name: {name} -  id: {idx}')


name: جمعه مباركه  -  id: {'$oid': '64761e0b9fa94c1d12f9bd1e'}
name: البيع ببغاء زنجباري افريقي ( ذكر +انثي )للاستفسار 01118589942 -  id: {'$oid': '647618039fa94c1d12f9ba40'}
name: صباح الخير  -  id: {'$oid': '647618039fa94c1d12f9ba2d'}
name: باندا -  id: {'$oid': '647620a89fa94c1d12f9be1a'}
name: مظلة عربيات  -  id: {'$oid': '647741a0079df174e8aa14a5'}
name: Omniguard سبراى ضد الحشرات. الأمن على اليفك. -  id: {'$oid': '64764b669fa94c1d12f9c8c4'}
name: نتمنى لكم جميعاً سنة جديدة سعيدة!  -  id: {'$oid': '647618039fa94c1d12f9ba0d'}
name: احدث اشكال منظم الطعام و الشراب  -  id: {'$oid': '647618039fa94c1d12f9ba20'}
name:   يابلاش ب ٧ بحق فلاش    -  id: {'$oid': '64740005a31ae693dc2de66c'}
name: গাছে কাঁঠাল গোপে তেল। -  id: {'$oid': '64765c299fa94c1d12f9ccd2'}
name: سي دهب بالدلاية اللى عملتها حبيبي  -  id: {'$oid': '647620a89fa94c1d12f9be1b'}
name: রসমালাই।।। -  id: {'$oid': '64765c299fa94c1d12f9ccba'}
name: فيتا داى وكيكى درى فود -  id: {'$oid': '647618b29fa94c1d12f9baad'}
name: شامبو للق

In [15]:
# Save the embeddings
torch.save(name_embeddings, 'name_embeddings.pt')
